In [1]:
import m2mcluster as m2m

In [2]:
from amuse.lab import *
from amuse.units import nbody_system,units
from amuse.community.bhtree.interface import BHTree

import matplotlib.pyplot as plt
import numpy as np


In [3]:
#Setup a star cluster for artificial observation
ocluster=m2m.setup_star_cluster(N=1000, Mcluster = 1000 | units.MSun, Rcluster = 9. |units.parsec, W0=7.)

#Measure artifical cluster's density profile asssuming an equal number of stars per bin
orlower,orad,orupper,orho=m2m.density(ocluster,bins=True,bintype='num')

In [4]:
#Initialize an M2M Star cluster
#Specify number of iterations to run algorithm for
#Specify number of workers to be used by Nbody code
cluster=m2m.starcluster(number_of_iterations=100,number_of_workers=1)

In [5]:
#Add the "observed" cluster density profile as an observable
cluster.add_observable(orlower,orupper,orho,'density',2)

In [6]:
#Initialize a model star cluster will an initial guess as the observed cluster's properties
cluster.initialize_star_cluster(N=1000, Mcluster = 1000 | units.MSun, Rcluster = 3. |units.parsec, softening = 0.75 | units.parsec, W0=1.)

(<amuse.datamodel.particles.Particles at 0x13e7826a0>,
 <amuse.units.nbody_system.nbody_to_si at 0x13e80b0d0>)

In [7]:
#Plot initial positions
cluster.xy_plot(filename='xyplot0.png')

In [8]:
#Compare initial density profiles
cluster.rho_prof(filename='rhoplot0.png')

In [9]:
#Exectute the made to measure algorithm
outfile=open('logfile','w')
for i in range(0,cluster.number_of_iterations):
    #Initialize a new N-body simulation ever time step. 
    #In this example I use 1% of the cluster's dynamical time for the integration timeste
    cluster.initialize_gravity_code('BHTree', dt=0.01*cluster.tdyn)
    #Evolve the model cluster forward for 10% of its dynamical time
    cluster.evolve(tend=0.1*cluster.tdyn)
    #Run the M2M algorithm, which will adjust all of the stellar masses based on kernel function
    cluster.evaluate(kernel=None,m2mepsilon=1.0e-3,plot=False)
    #Compare the new model density profile to the observed one
    cluster.rho_prof(filename='%s.png' % str(i).zfill(5))
    #Centre the star cluster and find determine Nbody conversion scales for next integration
    cluster.reinitialize_star_cluster()

    outfile.write('%i %f\n' % (i,cluster.criteria))
outfile.close()

0  TIME UNITS:  0.244963929474 Myr
1  TIME UNITS:  0.230983366667 Myr
2  TIME UNITS:  0.241817519836 Myr
3  TIME UNITS:  0.25293838336 Myr
4  TIME UNITS:  0.264216147323 Myr
5  TIME UNITS:  0.277546785313 Myr
6  TIME UNITS:  0.295200789153 Myr
7  TIME UNITS:  0.316498248025 Myr
8  TIME UNITS:  0.334885505361 Myr
9  TIME UNITS:  0.359282437568 Myr
10  TIME UNITS:  0.385239167949 Myr
11  TIME UNITS:  0.414696306658 Myr
12  TIME UNITS:  0.446038400303 Myr
13  TIME UNITS:  0.4810430348 Myr
14  TIME UNITS:  0.517044461388 Myr
15  TIME UNITS:  0.549526836682 Myr
16  TIME UNITS:  0.594922663424 Myr
17  TIME UNITS:  0.644736901363 Myr
18  TIME UNITS:  0.706493185734 Myr
19  TIME UNITS:  0.774456541245 Myr
20  TIME UNITS:  0.83596374553 Myr
21  TIME UNITS:  0.945440970167 Myr
22  TIME UNITS:  0.917567960582 Myr
23  TIME UNITS:  1.00290212856 Myr
24  TIME UNITS:  1.01314015729 Myr
25  TIME UNITS:  1.1815245514 Myr
26  TIME UNITS:  1.27455778488 Myr
27  TIME UNITS:  1.54751402867 Myr
28  TIME UNI